## Importing required libraries.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## Web-scraping the wikipedia page for the list of neighborhoods.

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India').text
soup = BeautifulSoup(data, 'html.parser')

In [3]:
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [4]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df.head()

,Neighborhood
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adibatla


In [5]:
df.shape

(200, 1)

## Getting the geographical coordinates of the neighborhoods.

In [6]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [7]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [9]:
coords

[[17.411200000000065, 78.50824000000006],
 [17.393000949133675, 78.45689980427697],
 [17.337650000000053, 78.56414000000007],
 [17.389800000000037, 78.47658000000007],
 [17.235790000000065, 78.54132000000004],
 [17.410610000000077, 78.51513000000006],
 [17.37751000000003, 78.48005000000006],
 [17.38738496982723, 78.46699458034638],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.535430000000076, 78.54427000000004],
 [17.385820000000024, 78.51836000000003],
 [17.53332000000006, 78.32529000000005],
 [17.435350000000028, 78.44861000000003],
 [17.45787000000007, 78.53882000000004],
 [17.40784000000002, 78.49150000000003],
 [17.385140000000035, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.40710000000007, 78.50233000000003],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.39931000000007, 78.49964000000006],
 [17.339920000000063, 78.54553000000004],
 [

In [8]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [9]:
df_coords.head()

,Latitude,Longitude
0,17.411200,78.50824
1,17.393001,78.45690
2,17.337650,78.56414
3,17.389800,78.47658
4,17.235790,78.54132


In [10]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df.head()

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. S. Rao Nagar,17.411200,78.50824
1,A.C. Guards,17.393001,78.45690
2,Abhyudaya Nagar,17.337650,78.56414
3,Abids,17.389800,78.47658
4,Adibatla,17.235790,78.54132


## Visualizing the neighborhoods in a map.

In [11]:
# get the coordinates of Hyderabad.
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="hyderabad_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.38878595, 78.46106473453146.


In [12]:
# create map of hyderabad
map_hy = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_hy)  
    
map_hy

## Getting the venues data from Foursquare API.

In [13]:
CLIENT_ID = 'H4CIAGBPDX3OYCQSHYAYBHWB24VDX4QCL103VSNA5SAGPAGY'
CLIENT_SECRET = 'KHAN5KFIEA3AYNIASHHFWNFDFL4LSMM3OI1MQVMQ5FZHPWBU'
VERSION = '20200815'

print('Foursquare credentails:')
print('Client ID: ' + CLIENT_ID)
print('Client secret: ' + CLIENT_SECRET)

Foursquare credentails:
Client ID: H4CIAGBPDX3OYCQSHYAYBHWB24VDX4QCL103VSNA5SAGPAGY
Client secret: KHAN5KFIEA3AYNIASHHFWNFDFL4LSMM3OI1MQVMQ5FZHPWBU


In [15]:
radius = 2000
limit = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        limit)
    
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [16]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6480, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. S. Rao Nagar,17.4112,78.50824,Bawarchi,17.406369,78.497662,Indian Restaurant
1,A. S. Rao Nagar,17.4112,78.50824,Devi 70 MM,17.406329,78.495409,Movie Theater
2,A. S. Rao Nagar,17.4112,78.50824,Sudharshan Theatre 35mm,17.406530,78.495150,Movie Theater
3,A. S. Rao Nagar,17.4112,78.50824,Subway,17.404173,78.514950,Sandwich Place
4,A. S. Rao Nagar,17.4112,78.50824,Baskin-Robbins,17.404311,78.510034,Ice Cream Shop


In [17]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. S. Rao Nagar,21,21,21,21,21,21
A.C. Guards,51,51,51,51,51,51
Abhyudaya Nagar,9,9,9,9,9,9
Abids,80,80,80,80,80,80
Adikmet,20,20,20,20,20,20
Afzal Gunj,50,50,50,50,50,50
Aghapura,59,59,59,59,59,59
"Aliabad, Hyderabad",9,9,9,9,9,9
Alijah Kotla,19,19,19,19,19,19


## Filtering venues for "Multiplex".

In [18]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 178 uniques categories.


In [19]:
venues_df['VenueCategory'].unique()

array(['Indian Restaurant', 'Movie Theater', 'Sandwich Place',
       'Ice Cream Shop', 'Coffee Shop', 'Convenience Store', 'Café',
       'Asian Restaurant', 'Train Station', 'Chinese Restaurant',
       'Bakery', 'Hyderabadi Restaurant', 'Juice Bar', 'Lounge',
       'South Indian Restaurant', 'Park', 'Bistro',
       'Middle Eastern Restaurant', 'Science Museum', 'Snack Place',
       'Vegetarian / Vegan Restaurant', 'Stadium', 'Pizza Place',
       'Performing Arts Venue', 'Hotel', 'Hotel Bar',
       'Fast Food Restaurant', 'Mobile Phone Shop', 'Fried Chicken Joint',
       'Department Store', 'Hookah Bar', 'Electronics Store',
       'Clothing Store', 'Paper / Office Supplies Store', 'Restaurant',
       'Food Truck', 'Shoe Store', 'Diner', 'Chaat Place', 'Neighborhood',
       'Burger Joint', 'Dessert Shop', 'Smoke Shop', 'Breakfast Spot',
       'Bar', 'Shopping Mall', 'Food', 'Multiplex', 'Gaming Cafe',
       'Indie Movie Theater', 'Farmers Market', 'Food Court',
       'Bus 

In [20]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [21]:
# one hot encoding
df_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

print(df_onehot.shape)
df_onehot.head()

(6480, 179)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Castle,Chaat Place,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Optical Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Rajasthani Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
df_grouped = df_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(df_grouped.shape)
df_grouped

(198, 179)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Castle,Chaat Place,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Optical Shop,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Rajasthani Restaurant,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.047619,0.000,0.000000,0.047619,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.047619,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.142857,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.095238,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.047619,

In [23]:
len(df_grouped[df_grouped["Multiplex"] > 0])

62

In [24]:
df_mall = df_grouped[["Neighborhoods","Multiplex"]]
df_mall.head()

,Neighborhoods,Multiplex
0,A. S. Rao Nagar,0.0000
1,A.C. Guards,0.0000
2,Abhyudaya Nagar,0.0000
3,Abids,0.0125
4,Adikmet,0.0000


## Clustering the neighborhoods.

In [25]:
# set number of clusters
kclusters = 3

df_clustering = df_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
df_merged = df_mall.copy()

# add clustering labels
df_merged["Cluster Labels"] = kmeans.labels_

In [27]:
df_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
df_merged.head()

,Neighborhood,Multiplex,Cluster Labels
0,A. S. Rao Nagar,0.0000,0
1,A.C. Guards,0.0000,0
2,Abhyudaya Nagar,0.0000,0
3,Abids,0.0125,0
4,Adikmet,0.0000,0


In [28]:
df_merged = df_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(df_merged.shape)
df_merged.head() # check the last columns!

(198, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.0000,0,17.411200,78.50824
1,A.C. Guards,0.0000,0,17.393001,78.45690
2,Abhyudaya Nagar,0.0000,0,17.337650,78.56414
3,Abids,0.0125,0,17.389800,78.47658
4,Adikmet,0.0000,0,17.410610,78.51513


In [29]:
# sort the results by Cluster Labels
print(df_merged.shape)
df_merged.sort_values(["Cluster Labels"], inplace=True)
df_merged

(198, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
124,Mallapur,0.000000,0,17.450023,78.609348
125,Mallepally,0.000000,0,17.391860,78.459050
126,Manikonda,0.000000,0,17.401390,78.391630
127,Marredpally,0.000000,0,17.447770,78.508730
129,Meerpet–Jillelguda,0.000000,0,17.329640,78.533030
130,"Mehboob ki Mehendi, Hyderabad",0.000000,0,17.362015,78.470795
131,Mehdipatnam,0.000000,0,17.392630,78.442190
132,Mettuguda,0.000000,0,17.427740,78.528920
120,Mahatma Gandhi Road (Secunderabad),0.000000,0,17.428548,78.487851


## Visualizing the clusters in a map.

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [33]:
df_merged.loc[df_merged['Cluster Labels'] == 0]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
124,Mallapur,0.000000,0,17.450023,78.609348
125,Mallepally,0.000000,0,17.391860,78.459050
126,Manikonda,0.000000,0,17.401390,78.391630
127,Marredpally,0.000000,0,17.447770,78.508730
129,Meerpet–Jillelguda,0.000000,0,17.329640,78.533030
130,"Mehboob ki Mehendi, Hyderabad",0.000000,0,17.362015,78.470795
131,Mehdipatnam,0.000000,0,17.392630,78.442190
132,Mettuguda,0.000000,0,17.427740,78.528920
120,Mahatma Gandhi Road (Secunderabad),0.000000,0,17.428548,78.487851


In [32]:
df_merged.loc[df_merged['Cluster Labels'] == 1]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
14,Anandbagh,0.090909,1,17.457870,78.538820
91,Karwan,0.058824,1,17.379070,78.436680
190,Sanghi Nagar,0.060000,1,17.402038,78.457069
192,Saroornagar,0.071429,1,17.354420,78.539210
185,Safilguda,0.071429,1,17.466430,78.535650
186,Saidabad,0.066667,1,17.363570,78.509180
93,Khairtabad,0.080000,1,17.405920,78.458560
175,Raj Bhavan Road,0.080000,1,17.413475,78.459830
187,Saifabad,0.060606,1,17.405390,78.471380
181,"Red Hills, Hyderabad",0.076087,1,17.397410,78.462740


In [34]:
df_merged.loc[df_merged['Cluster Labels'] == 2]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
97,Kompally,0.222222,2,17.53318,78.48102
81,Jalal Baba Nagar,0.285714,2,17.35442,78.43255
40,Champapet,0.200000,2,17.34767,78.51552
